# Break large collection into smaller collection based on week number
## 1. connect mongodb

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [2]:
#print(client.database_names())
db_twitter = client["Twitter"]
collections = db_twitter.collection_names()

In [3]:
dic_collection={}
for i in collections:
    dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())
    
for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2019_W8_Twitter_Australia: 198
2019_W8_Twitter_Other: 31,057
2019_W9_Twitter_Australia: 2,316
2019_W9_Twitter_Other: 711,083
twitter-australia: 460,138
twitter-other: 249,447,870
twitter-richard-2014: 211,755
twitter-richard-2015: 198,258
twitter-richard-2016: 10,801
twitter-richard-2017: 21,797,263
twitter-richard-2018: 10,679,787


## 2. split twitter-australia collection based on week number in 2018

In [ ]:
list_2018 = ["1545609600000","1546214400000"]    # get timestamp_ms of 2018 week 51 and week 52

for i in list_2018: 
    w = int((int(i) - 1514764800000)/1000/604800)       # calculate week number
    
    # create pipeline to search record with timestamp_ms which is less than i
    pipeline = [ 
        {"$match": {'timestamp_ms':{'$lt':i}}},
        {"$out": "2018_W" + str(w) + "_Twitter_Other"},
    ]
    
    # prevent emptying the collection if the method is already done before or some errors occur during the copying process
    try:
        if db_twitter['twitter-other'].find_one({'timestamp_ms':{'$lt':i}}) == None:
            print("No document is found")
            break
        db_twitter['twitter-other'].aggregate(pipeline)
    except:
        print("error")
        break
        
    # remove records in the old collection after previous part finish
    db_twitter['twitter-other'].remove({'timestamp_ms':{'$lt':i}}) 
    
    print("finised 2018 week "+ str(w))

## 3. split twitter-australia collection based on week number in 2019

In [ ]:
# get timestamp_ms of 2019 from week 1 to week 8
list_2019 = ["1546819200000", "1547424000000","1548028800000","1548633600000","1549238400000","1549843200000","1550448000000","1551052800000"]

for i in list_2019:
    w = int((int(i) - 1546214400000)/1000/604800)      # calculate week number
    
    # create pipeline to search record with timestamp_ms which is less than i
    pipeline = [ 
        {"$match": {'timestamp_ms':{'$lt':i}}},
        {"$out": "2019_W" + str(w) + "_Twitter_Other"},
    ]
    
    # prevent emptying the collection if the method is already done before or some errors occur during the copying process
    try:
        if db_twitter['twitter-other'].find_one({'timestamp_ms':{'$lt':i}}) == None:
            print("No document is found")
            break
        db_twitter['twitter-other'].aggregate(pipeline)
    except:
        print("error")
        break
        
    # remove records in the old collection after previous part finish
    db_twitter['twitter-other'].remove({'timestamp_ms':{'$lt':i}}) 
    
    print("finised 2019 week "+ str(w))